# The Data

This notebook explains the data for this project.  All NBA games from 1951 to 2020.  For each game, the score of each team, the home/away teams, the number of overtimes and whether or not it was a playoff game.  There are 63,863 games collected.  

## Source of the Data

All of the game data was collected from [basketball-reference.com](https://www.basketball-reference.com). Because these are in nested webpages, a script was built to scrape the website and download the data. 

### The Script

The available julia script `parse-NBA-data.jl` was written to aid in the collection.  This script has a number of options, but in short fetches data for a given span of years and then either writes the data to a sqlite database (recommended) or to CSV files.  Here are the options on the script

```
USAGE: julia parse-NBA-data.jl [--help] 

    --create-csv:
        Rebuild the CSV files

    --csv-dir: String
        Directory to store the CSV files

    --db-name: String
        Database file name

    --html-dir: String
        Directory where the raw HTML files are.

    --rebuild-db:
        rebuild the SQLite database

    --verbose:
        Turn on verbosity

    --years: [Ints]
        Years for the rebuild
```

#### Working with a script

If you haven't done much with any kind of script, you'll need to open a terminal window (Terminal on MacOS or PowerShell on Windows).  If you are running jupyter lab, you can access one of these by clicking the launcher (+ button in the upper left corner) then select "terminal" on the lower row of buttons. 

You should see a prompt.  An example on MacOS is:
```
➜  bb-streaks git:(main) ✗  
```

Hopefully the launcher puts you in the correct directory. For all of these scripts you should be in the top of the `bb-streaks` directory.  If not you may need to change directory (`cd`) to the correct directory. 


#### Fetching the Data

To fetch the data for some given years (1960 to 1969) and store into the folder `nba-raw-html`, 
```bash
julia parse-NBA-data.jl --verbose --html-dir=nba-raw-html --years=1960..1969 --fetch-data
```

This will take a bit of time mainly due to internet connection speed.  Also, the site `basketball-reference.com` has a limit of number of requests made.  From practice you can download about 5-10 years of data before you get an error.  It appears that you'll need to wait another to download another batch. 


#### Putting the data into CSV files

In order to effectively use the data, the data must be converted from HTML files to either CSV files or to a sqlite database (recommended).  Running scripts will be faster using the latter method.  The advantage to the CSV files is that they can be examined using any text editor or spreadsheet programs like Microsoft Excel. 

To create CSV files for the 1955 to 2020 seasons and place the results in the directory `nba_csv`, enter
```bash
julia parse-NBA-data.pl --verbose --html-dir=nba-raw-html --years=1955..2020 --create_csv --csv_dir=nba_csv
```

This requires that all of the data is in the directory `nba-raw-html` or you will get an error. 

#### Putting the data into a database

An alternative to putting the data into CSV files, you can put all of the data in a database and this script uses a SQLite database.  Note: you will need to download the freely available `sqlite` for your platform. 

To put all seasons 1955 to 2020 into the database stored as `nba.sqlite`, enter
```bash
julia parse-nba-data.jl --verbose --rebuild-db --database-name=nba.sqlite --html-dir nba-raw-html-new --years=1955..2020
```

This takes about 15 minutes or so to run completely.  It is parsing hundreds of HTML files which is not a trivial thing to do and then puts the data into a database. 

Note: although it is a little more complicated to use a database, it is recommended in that the desired data can be easily pulled out of the database.  Also, there are functions to do this in the `NBA.jl` module, and another notebook is provided to explain those functions.

### Using CSV files

To load in a CSV file, you'll need to load the `CSV` package (and may need to install it if you get an error).  We'll use the `DataFrames` and `Chain` packages as well.  You should also read the `getting_started` notebook before this.

Note, the `ENV["DATAFRAMES_ROWS"]=10` tells julia to just show 10 rows of a dataframe.

In [1]:
using CSV, DataFrames, Chain
ENV["DATAFRAMES_ROWS"] = 10

10

And then the following reads in a file and outputs it as a `DataFrame`.  This is the entire 2020 season.  Note: relative to this jupyter notebook, the data is up a directory (..) and then in the `nba_csv` directory. 

In [4]:
nba2020 = CSV.read("../nba_csv/2020.csv", DataFrame)

Row,DATE,SEASON,VISITOR_TEAM,VISITOR_SCORE,HOME_TEAM,HOME_SCORE,OTS,PLAYOFF
,Date,Int64,String31,Int64,String31,Int64,Int64,String1
1,2019-10-22,2020,New Orleans Pelicans,122,Toronto Raptors,130,1,N
2,2019-10-22,2020,Los Angeles Lakers,102,Los Angeles Clippers,112,0,N
3,2019-10-23,2020,Chicago Bulls,125,Charlotte Hornets,126,0,N
4,2019-10-23,2020,Detroit Pistons,119,Indiana Pacers,110,0,N
5,2019-10-23,2020,Cleveland Cavaliers,85,Orlando Magic,94,0,N
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
1139,2020-10-02,2020,Miami Heat,114,Los Angeles Lakers,124,0,Y
1140,2020-10-04,2020,Los Angeles Lakers,104,Miami Heat,115,0,Y
1141,2020-10-06,2020,Los Angeles Lakers,102,Miami Heat,96,0,Y


If we want the regular season for one team, we will filter with the subset function (See the `getting_started` notebook for more explanation):

In [7]:
@chain nba2020 begin
  subset([:VISITOR_TEAM, :HOME_TEAM, :PLAYOFF] => (a,h,p) -> (a .== "Miami Heat" .|| h .== "Miami Heat") .&& p .== "N")
end

Row,DATE,SEASON,VISITOR_TEAM,VISITOR_SCORE,HOME_TEAM,HOME_SCORE,OTS,PLAYOFF
,Date,Int64,String31,Int64,String31,Int64,Int64,String1
1,2019-10-23,2020,Memphis Grizzlies,101,Miami Heat,120,0,N
2,2019-10-26,2020,Miami Heat,131,Milwaukee Bucks,126,1,N
3,2019-10-27,2020,Miami Heat,109,Minnesota Timberwolves,116,0,N
4,2019-10-29,2020,Atlanta Hawks,97,Miami Heat,112,0,N
5,2019-10-31,2020,Miami Heat,106,Atlanta Hawks,97,0,N
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
69,2020-08-06,2020,Miami Heat,116,Milwaukee Bucks,130,0,N
70,2020-08-08,2020,Phoenix Suns,119,Miami Heat,112,0,N
71,2020-08-10,2020,Indiana Pacers,92,Miami Heat,114,0,N


which gets the regular season games for the Miami Heat in 2020.  Miami only had 73 regular season games in the shorted 2020 season due to COVID.

Although this seems fairly straightforward, if you use CSV files, each season is in a separate file and combining over multiple seasons can be burdensome.  Therefore, for this project we used a database and the simplest data is a SQLITE as we will need in the next section.

### Getting the Data into a SQLITE database

Although CSV files are well supported and are more familiar to most people, it is advantageous to use a database to store the data.  The reasons are many, but speed, storage size are among two of them.  The script `parse-NBA-data.pl` has an option to fill a sqlite database with the data in a certain format.  Additionally, there are many Julia functions in the module NBA.jl to retrieve data from the database.  Another file will go over these.

We will use the module/package SQLite to access the database directly.  Note: you may need to download/install this before using it.

In [8]:
using SQLite

The following will load the database that is store in the file `nba.sqlite` in the folder above the current one.

In [9]:
db = SQLite.DB("../nba.sqlite")

SQLite.DB("../nba.sqlite")

The following will load the 2000 season:

In [10]:
nba2000db = DataFrame(DBInterface.execute(db,"SELECT * FROM NBA_SCORES JOIN NBA_GAME_DATES on NBA_SCORES.DATE_ID= NBA_GAME_DATES.DATE_ID WHERE SEASON=2000"))

Row,GAME_ID,DATE_ID,VISITOR_ID,VISITOR_SCORE,HOME_ID,HOME_SCORE,OTS,PLAYOFF,DATE_ID_1,DATE,SEASON
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,String,Int64,String,Int64
1,35469,35469,70937,86,70938,100,0,N,35469,1999-11-02,2000
2,35470,35469,70939,96,70940,108,0,N,35469,1999-11-02,2000
3,35471,35469,70941,102,70942,107,1,N,35469,1999-11-02,2000
4,35472,35469,70943,98,70944,93,0,N,35469,1999-11-02,2000
5,35473,35469,70945,104,70946,92,0,N,35469,1999-11-02,2000
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
1260,36728,36728,70949,104,70957,111,0,Y,36728,2000-06-09,2000
1261,36729,36729,70957,91,70949,100,0,Y,36729,2000-06-11,2000
1262,36730,36730,70957,120,70949,118,1,Y,36730,2000-06-14,2000


The string that fetches out of the database is a bit complicated in that we chose to split the database up into different tables for efficiency.  To pull a season out, we need to join two tables, then filter (with the `WHERE` command).  

As you may notice, there are columns labelled `_ID` which are ids to games, dates, teams, not directly the strings representing them. 

If we have all of the data in a sqlite database, we can do any database calls to extract or manipulate the database.  For example, instead of using team ids and game ids, we can join the other tables to make a more readable table.  For example: 


In [11]:
nba2000reg = DataFrame(DBInterface.execute(db,
    """SELECT NBA_GAME_DATES.DATE, NBA_GAME_DATES.SEASON, VTN.NAME AS VISITOR_TEAM, VISITOR_SCORE, HTN.NAME AS HOME_TEAM, HOME_SCORE, OTS, PLAYOFF FROM NBA_SCORES 
      JOIN NBA_GAME_DATES ON NBA_SCORES.DATE_ID = NBA_GAME_DATES.DATE_ID 
      JOIN NBA_TEAM_NAMES as VTN ON NBA_SCORES.VISITOR_ID = VTN.TEAM_ID
      JOIN NBA_TEAM_NAMES AS HTN ON NBA_SCORES.HOME_ID = HTN.TEAM_ID
      WHERE NBA_GAME_DATES.SEASON=2000"""))

Row,DATE,SEASON,VISITOR_TEAM,VISITOR_SCORE,HOME_TEAM,HOME_SCORE,OTS,PLAYOFF
,String,Int64,String,Int64,String,Int64,Int64,String
1,1999-11-02,2000,Orlando Magic,86,Charlotte Hornets,100,0,N
2,1999-11-02,2000,Golden State Warriors,96,Dallas Mavericks,108,0,N
3,1999-11-02,2000,Phoenix Suns,102,Denver Nuggets,107,1,N
4,1999-11-02,2000,Milwaukee Bucks,98,Houston Rockets,93,0,N
5,1999-11-02,2000,Seattle SuperSonics,104,Los Angeles Clippers,92,0,N
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
1260,2000-06-09,2000,Indiana Pacers,104,Los Angeles Lakers,111,0,Y
1261,2000-06-11,2000,Los Angeles Lakers,91,Indiana Pacers,100,0,Y
1262,2000-06-14,2000,Los Angeles Lakers,120,Indiana Pacers,118,1,Y


You probably agree that writing the commands to create the 2000 season was a bit crazy and we won't have to do this. 

As we will see in the next jupyter notebook, many of the standard database calls have been put into functions for ease.  Note that once we have a DataFrame though, we can maninpulate it using many of the techniques we saw in the getting_started notebook.  

The following pulls just the regular season

In [12]:
@chain nba2000reg begin
  subset(:PLAYOFF => p-> p .== "N")
end

Row,DATE,SEASON,VISITOR_TEAM,VISITOR_SCORE,HOME_TEAM,HOME_SCORE,OTS,PLAYOFF
,String,Int64,String,Int64,String,Int64,Int64,String
1,1999-11-02,2000,Orlando Magic,86,Charlotte Hornets,100,0,N
2,1999-11-02,2000,Golden State Warriors,96,Dallas Mavericks,108,0,N
3,1999-11-02,2000,Phoenix Suns,102,Denver Nuggets,107,1,N
4,1999-11-02,2000,Milwaukee Bucks,98,Houston Rockets,93,0,N
5,1999-11-02,2000,Seattle SuperSonics,104,Los Angeles Clippers,92,0,N
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
1185,2000-04-19,2000,Toronto Raptors,100,Orlando Magic,106,0,N
1186,2000-04-19,2000,Denver Nuggets,96,Portland Trail Blazers,95,0,N
1187,2000-04-19,2000,Los Angeles Lakers,98,San Antonio Spurs,103,1,N


And the following is the regular season games for the Charlotte Hornets:

In [13]:
@chain nba2000reg begin
  subset([:PLAYOFF, :VISITOR_TEAM, :HOME_TEAM] => (p,v,h)-> p .== "N" .&& (v .== "Charlotte Hornets" .|| h .== "Charlotte Hornets"))
end

Row,DATE,SEASON,VISITOR_TEAM,VISITOR_SCORE,HOME_TEAM,HOME_SCORE,OTS,PLAYOFF
,String,Int64,String,Int64,String,Int64,Int64,String
1,1999-11-02,2000,Orlando Magic,86,Charlotte Hornets,100,0,N
2,1999-11-04,2000,Indiana Pacers,89,Charlotte Hornets,98,0,N
3,1999-11-05,2000,Charlotte Hornets,100,Boston Celtics,103,0,N
4,1999-11-07,2000,Charlotte Hornets,99,Toronto Raptors,109,0,N
5,1999-11-10,2000,Milwaukee Bucks,111,Charlotte Hornets,117,0,N
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
78,2000-04-12,2000,Atlanta Hawks,87,Charlotte Hornets,119,0,N
79,2000-04-14,2000,Charlotte Hornets,109,Milwaukee Bucks,106,1,N
80,2000-04-16,2000,Charlotte Hornets,105,Boston Celtics,102,0,N


Since the sql commands to pull info out of the databases is a bit complicated, these will be placed into a separate module that will make it easy to pull out.  See the `NBA_module` notebook for help getting started with this.